## Árvore B 

O código abaixo ilustra a inclusão, busca e percursos em árvores B, com grau máximo = 4. Por isso, também pode ser usada como implementação de uma **árvore 2-3-4**.

Esta versão não possui acesso ao disco, para permitir testes online usando o Notebook.

In [1]:
#include <iostream>
#include <stdlib.h> 
using namespace std;

#define GRAU_MAXIMO 4
#define GRAU_MINIMO 2

Estrutura para uma árvore B simples. A árvore tem um conjunto de chaves e de nós filhos. O número de nós filhos é sempre 1 unidade maior que o número de chaves.

In [2]:
struct tNo {
  int chave [GRAU_MAXIMO-1];
  tNo *p [GRAU_MAXIMO];
  int tam; 
} 

Função para inicialização do nó, com alocação de memória e atribuição de valores NULL para os ponteiros e 0 para o tamanho.

In [3]:
tNo *criaNo (){
    tNo *n = (tNo *)malloc (sizeof (tNo));
    n->tam = 0;
    for (int i=0; i < GRAU_MAXIMO ;i++)
       n->p[i] = NULL;
    return n;
}

Função que lê um token, separado por espaço, e converte para um número inteiro.

In [4]:
int token_to_num(const char *str, int *indice){    
    char token[100];
    int i = 0;
    while (str[*indice] != '\0' && str[*indice] != ' '){
        token[i] = str[*indice];
        i++;
        (*indice)++;
    }
    token[i] = '\0';
    (*indice)++;    
    return atoi(token);
}

Percurso da árvore em **PRÉ-ORDEM** em uma árvore B, pela chave.

In [5]:
void preordem(tNo *no){
   if (no != NULL) {
       for (int i=0; i < no->tam; i++){
           cout << no->chave[i] << '.';
           preordem(no->p[i]);
       }
       preordem(no->p[no->tam]);
   }
}


Percurso da árvore em **ORDEM** pela chave.

In [6]:
void emordem(tNo *no){    
   if (no != NULL) {
       for (int i=0; i < no->tam; i++){
           emordem(no->p[i]);
           cout << no->chave[i] << '.';
       }
       emordem(no->p[no->tam]);
   }
}

Percurso da árvore em **PÓS-ORDEM**.

In [7]:
void posordem(tNo * no){
   if (no != NULL) {
       for (int i=0; i < no->tam; i++){
           posordem(no->p[i]);
           if (i-1 >=0)
               cout << no->chave[i-1] << '.';
       }       
       posordem(no->p[no->tam]);     
       cout << no->chave[no->tam-1] << '.';
   }
}

Conta o número de nós da árvore.

In [8]:
int contaNos(tNo *no){
   int nos = 0; 
   if (no != NULL) {
       for (int i=0; i < no->tam; i++){           
           nos += contaNos(no->p[i]);
       }
       nos += contaNos(no->p[no->tam]);
       nos += 1;
   }
   return nos;
}

Uma árvore B pode ter mais de 1 nó por chave. Esta função conta o total de todas as chaves em todos os nós.

In [9]:
int contaChaves(tNo *no){
   int chaves = 0; 
   if (no != NULL) {
       for (int i=0; i < no->tam; i++){           
           chaves += contaChaves(no->p[i]) + 1;
       }
       chaves += contaChaves(no->p[no->tam]);
   }
   return chaves;
}

Calcula a altura da árvore. Como a árvore B é sempre balanceada, a contagem da altura é feita descendo pelo primeiro ponteiro à esquerda até o nó folha.

In [10]:
int altura (tNo *p) {
    int i = 0;
    while (p->p[0] != NULL){
        p = p->p[0];
        i ++;
    }
    return i;
}

Função que divide um nó de uma árvore B, usada na inclusão, quando o nó está cheio. Um nó está cheio quando o número de chaves é igual ao grau máximo - 1.

In [11]:
void splitNo (tNo *pai, int indice, tNo *no) {
    tNo *novo = criaNo();
    novo->tam = GRAU_MINIMO - 1;
    //atribui chaves da segunda metade para o novo nó
    for (int i=0; i < GRAU_MINIMO-1; i++){
        novo->chave[i] = no->chave[i+GRAU_MINIMO];
    }
    //atribui os pointeiros da segunda metade, se não é folha
    if (no->p[0]  != NULL) {//novo->chave[i]
        for (int i=0; i <= GRAU_MINIMO-1; i++)
            novo->p[i] = no->p[i + GRAU_MINIMO];
    }
    no->tam = GRAU_MINIMO - 1;
    //desloca os ponteiros do nó pai para a direita
    for (int i = pai->tam;  i > indice; i--)
        pai->p[i+1] = pai->p[i];
    //ultimo indice aponta para o novo nó
    pai->p[indice+1] = novo;
    //desloca as chaves do pai para a direita
    for (int i = pai->tam-1; i >= indice; i--)
        pai->chave[i+1] = pai->chave[i];
    //sobe a chave
    pai->chave[indice] = no->chave[GRAU_MINIMO-1];
    pai->tam = pai->tam +1;
}

Função que faz o split da árvore B na raiz. É diferente do split dos demais nós, pois são criados 2 novos nós: a nova raiz e o novo nó à direita.

In [12]:
tNo *splitRaiz(tNo * raiz){
    tNo *novaRaiz = criaNo();
    splitNo(novaRaiz, 0, raiz);
    novaRaiz->p[0]=raiz;
    novaRaiz->p[0]->tam = GRAU_MINIMO - 1;
    return novaRaiz;
}

Funcão para inclusão de novo nó na árvore B, dado um nó raiz. O split é feito na descida, isto é, sempre que encontrar um nó cheio, irá chamar a função _splitNo_, e continuará a descida.

In [13]:
tNo* inclui (tNo *no, int chave, tNo* raiz){
    int i = no->tam -1;
    tNo *filho;
    if (no->tam == GRAU_MAXIMO - 1 && no == raiz) {//split na raiz, necessita de tratamento específico
        raiz = splitRaiz(raiz);
        no = raiz;
        i = no->tam - 1;
    }   
    if (no->p[0] == NULL){ //encontrou um nó folha para inclusão
        while (i >= 0 && chave < no->chave[i]) {
            no->chave[i+1] = no->chave[i];
            i--;
        }
        no->chave[i+1] = chave;
        no->tam = no->tam+1;
    } 
    else { //desce na árvore e faz o split se necessário
        while (i >= 0 && chave < no->chave[i])
            i--;
        i++;
        filho = no->p[i];
        if (filho->tam == GRAU_MAXIMO - 1){
            splitNo(no, i, filho);
            if (chave > no->chave[i]) 
                i ++;
        }
        inclui( no->p[i], chave, raiz);    
    }
    return raiz;
}

In [14]:
tNo* montaarvore(const char *str){
    tNo *raiz = NULL;
    int i = 0, v =0;
    raiz = criaNo();
    while (str[i]!='\0')
        raiz = inclui (raiz, token_to_num(str, &i), raiz);
    return raiz;        
}


In [15]:
void imprime(const char *str, tNo *no){
    cout << "||" << str << " ";
    if (no!=NULL) {
        cout << "[";
        for(int i=0; i < no->tam; i++) 
            cout << no->chave[i] << '.';
    }
    else cout << "não";
    cout << "] encontrado\n";
}

Busca em árvore B. Percorre todos os nós e suas chaves.

In [16]:
tNo *busca (tNo *no, int chave) {
    if( no != NULL ){ 
        for(int i=0; i < no->tam; i++) {
            if( no->chave[i] == chave ) 
                return no; 
            else 
                if( chave < no->chave[i] ) 
                    return busca( no->p [i], chave ); 
        }
        return busca( no->p[ no->tam ], chave ); 
    }
    else
        return NULL;
}

In [17]:
void iniciaprograma(){
    tNo *raiz = NULL;
    
    //char entrada[] = "20 13 17 22 11 23 12 9\0";
    char entrada[] = "20 13 17 22 11 23 12 9 25 21 18 19\0";
    raiz=montaarvore(entrada); 
    
    cout << "\nPercurso em pré-ordem: ";
    preordem(raiz); cout << "\n";
    cout << "Percurso em ordem: ";
    emordem(raiz); cout << "\n";
    cout << "Percurso em pós-ordem: ";
    posordem(raiz);
    cout << "\nTotal de nós: " << contaNos(raiz);
    cout << "\nTotal de chaves: " << contaChaves(raiz);
    cout << "\nAltura da árvore: " << altura(raiz) << "\n\n";
    
    imprime (" busca ", busca(raiz, 13));
    imprime (" busca ", busca(raiz, 22));
    imprime (" busca ", busca(raiz, 17));
    imprime (" busca 15 ", busca(raiz, 15));
    imprime (" busca ", busca(raiz, 23));
    imprime (" busca ", busca(raiz, 21));
    imprime (" busca ", busca(raiz, 18));
    
    
}

In [18]:
iniciaprograma();


Percurso em pré-ordem: 17.12.9.11.13.20.18.19.22.21.23.25.
Percurso em ordem: 9.11.12.13.17.18.19.20.21.22.23.25.
Percurso em pós-ordem: 9.11.13.12.18.19.21.20.23.25.22.17.
Total de nós: 8
Total de chaves: 12
Altura da árvore: 2

|| busca  [13.] encontrado
|| busca  [20.22.] encontrado
|| busca  [17.] encontrado
|| busca 15  não] encontrado
|| busca  [23.25.] encontrado
|| busca  [21.] encontrado
|| busca  [18.19.] encontrado
